# probit-регрессия: LR-тест, совместная значимость

In [5]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from scipy.stats import chi2 # критические значения chi2-распределения

# Не показывать FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
# импорт датасета
df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/econometrica/main/econometrica2/data-csv/TableF5-1.csv')
# импорт данных из локального файла
# df = pd.read_csv('TableF5-1.csv')
df

,LFP,WHRS,KL6,K618,WA,WE,WW,RPWG,HHRS,HA,HE,HW,FAMINC,MTR,WMED,WFED,UN,CIT,AX
0,1,1610,1,0,32,12,3.3540,2.65,2708,34,12,4.0288,16310,0.7215,12,7,5.0,0,14
1,1,1656,0,2,30,12,1.3889,2.65,2310,30,9,8.4416,21800,0.6615,7,7,11.0,1,5
2,1,1980,1,3,35,12,4.5455,4.04,3072,40,12,3.5807,21040,0.6915,12,7,5.0,0,15
3,1,456,0,3,34,12,1.0965,3.25,1920,53,10,3.5417,7300,0.7815,7,7,5.0,0,6
4,1,1568,1,2,31,14,4.5918,3.60,2000,32,12,10.0000,27300,0.6215,12,14,9.5,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,0,0,0,2,40,13,0.0000,0.00,3020,43,16,9.2715,28200,0.6215,10,10,9.5,1,5
749,0,0,2,3,31,12,0.0000,0.00,2056,33,12,4.8638,10000,0.7715,12,12,7.5,0,14
750,0,0,0,0,43,12,0.0000,0.00,2383,43,12,1.0898,9952,0.7515,10,3,7.5,0,4
751,0,0,0,0,60,12,0.0000,0.00,1705,55,8,12.4400,24984,0.6215,12,12,14.0,1,15


## Спецификация и подгонка модели
Для датасета `mroz_Greene` рассморим probit-регрессию **LFP на WA, WA^2, WE, KL6, K618, CIT, UN, log(FAMINC)**.

In [7]:
mod = smf.logit(formula='LFP~WA+I(WA**2)+WE+KL6+K618+CIT+UN+np.log(FAMINC)', data=df) # спецификация модели
res = mod.fit() # подгонка модели
res.nobs # число наблюдений, на которых была подогнана модель

Optimization terminated successfully.
         Current function value: 0.613860
         Iterations 5


753

## LR-тест: совместная значимость
Тестируем значимость влияния возраста жены, т.е. $$H_0:\beta_{WA}=\beta_{WA^2}=0.$$

Тестовая статистика $LR=2(\log L-\log L_{r})$ где 
- $\log L$ – лог-правдоподобие для исходной регрессии
- $\log L_{r}$ - лог-правдоподобие для регрессии с ограничениями (без переменных из гипотезы)

Для вычисления тестовой статистики нужно оценить две вложенных модели: без ограничений и с ограничениями

## <span style="color:#59afe1">**Важно**: обе регрессия д.б. оценены на одном и том же датасете! </span>

## Уровень значимости
Пусть уровень значимости $\alpha=10\%=0.1$

In [8]:
# подгонка модели с ограничениями
df_mod = df[['LFP', 'WA', 'WE', 'KL6', 'K618', 'CIT', 'UN', 'FAMINC']].dropna()
mod_r = smf.logit(formula ='LFP~WE+KL6+K618+CIT+UN+np.log(FAMINC)', data = df_mod)
res_r = mod_r.fit()
res_r.nobs # число наблюдений, на которых была подогнана модель

Optimization terminated successfully.
         Current function value: 0.631705
         Iterations 5


753

### Тестовая статистика LR-теста 

In [9]:
# Тестовая статистика LR-теста с округленим
lr_stat=2*(res.llf-res_r.llf)
lr_stat.round(3)

26.875

### Степени свободы для $\chi^2$-распределения 

In [10]:
res.df_model-res_r.df_model

2.0

### P-значение тестовой статистики LR-теста

In [11]:
# P-значение тестовой статистики LR-теста с округленим
lr_pvalue = chi2.sf(lr_stat, df=res.df_model-res_r.df_model)
lr_pvalue.round(3)

0.0

### Критическое значение $\chi_{df}^2(\alpha)$

In [12]:
sign_level = 0.1 # уровень значимости
chi2.ppf(q=1-sign_level, df=res.df_model-res_r.df_model).round(3) 

4.605

### Значимо ли возраста жены?

`влияние возраста жены значимо` $(LR>\chi_{df}^2(\alpha))$

In [14]:
# Статистика теста Вальда и её P-значение
res.wald_test('WA=I(WA ** 2)=0')

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=[[25.53332463]], p-value=2.854361687056113e-06, df_denom=2>